In [1]:
!pip install yacs
!pip install tensorboardX

In [2]:
import os
import shutil

# Crea la cartella dove l'API Kaggle si aspetta il file
os.makedirs('/root/.kaggle', exist_ok=True)

# Copia il file json dal dataset input nella cartella corretta
shutil.copy('/kaggle/input/api-kaggle/kaggle.json', '/root/.kaggle/kaggle.json')

# Imposta i permessi giusti
!chmod 600 /root/.kaggle/kaggle.json



In [3]:
import os

base_kaggle_path = "/kaggle/working/"
folder_names = [
    'data/loveda/train/',
    'data/loveda/val/',
    'data/list/loveda/rural',
    'data/list/loveda/urban_rural',
    'data/list/loveda/urban_urban',
    'pretrained_models/imagenet'
]

for relative_path in folder_names:
    full_path = os.path.join(base_kaggle_path, relative_path)
    if not os.path.exists(full_path):
        os.makedirs(full_path)
        print(f"Created: {full_path}")
    else:
        print(f"Already exists: {full_path}")



Already exists: /kaggle/working/data/loveda/train/
Already exists: /kaggle/working/data/loveda/val/
Already exists: /kaggle/working/data/list/loveda/rural
Already exists: /kaggle/working/data/list/loveda/urban_rural
Already exists: /kaggle/working/data/list/loveda/urban_urban
Already exists: /kaggle/working/pretrained_models/imagenet


In [4]:
import shutil
import os

list_src = [
    '/kaggle/input/loveda-splits/Train/Train/Rural',
    '/kaggle/input/loveda-splits/Train/Train/Urban',
    '/kaggle/input/loveda-splits/Val/Val/Rural',
    '/kaggle/input/loveda-splits/Val/Val/Urban'
]

list_dst = [
    '/kaggle/working/data/loveda/train/Rural',
    '/kaggle/working/data/loveda/train/Urban',
    '/kaggle/working/data/loveda/val/Rural',
    '/kaggle/working/data/loveda/val/Urban'
]

for src, dst in zip(list_src, list_dst):
    if not os.path.exists(dst):
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copytree(src, dst)
        print(f"Copied: {src} → {dst}")
    else:
        print(f"Skipped (already exists): {dst}")


Skipped (already exists): /kaggle/working/data/loveda/train/Rural
Skipped (already exists): /kaggle/working/data/loveda/train/Urban
Skipped (already exists): /kaggle/working/data/loveda/val/Rural
Skipped (already exists): /kaggle/working/data/loveda/val/Urban


In [5]:
import shutil
import os

dst = '/kaggle/working/configs/'

# Rimuove se esiste già
if os.path.exists(dst):
    shutil.rmtree(dst)

# Ora copia senza problemi
shutil.copytree('/kaggle/input/configs-pidnet/configs', dst)

'/kaggle/working/configs/'

In [6]:
import os
import shutil

# Crea la directory se non esiste
os.makedirs('/kaggle/working/pretrained_models/imagenet', exist_ok=True)

# Ora copia il file
shutil.copy('/kaggle/input/pidnet-pretrained/PIDNet_S_ImageNet.pth.tar', 
            '/kaggle/working/pretrained_models/imagenet')


'/kaggle/working/pretrained_models/imagenet/PIDNet_S_ImageNet.pth.tar'

In [7]:
shutil.copy('/kaggle/input/pidnet-pretrained/PIDNet_S_ImageNet.pth.tar', '/kaggle/working/pretrained_models/imagenet')

'/kaggle/working/pretrained_models/imagenet/PIDNet_S_ImageNet.pth.tar'

In [8]:
%cd '/kaggle/working'

/kaggle/working


# _init_paths.py

In [9]:
import os

def create_lst_file(image_dir, label_dir, output_lst):
    # List and sort files numerically
    images = sorted(os.listdir(image_dir), key=lambda x: int(os.path.splitext(x)[0]))
    labels = sorted(os.listdir(label_dir), key=lambda x: int(os.path.splitext(x)[0]))

    os.makedirs(os.path.dirname(output_lst), exist_ok=True)  # Ensure the directory exists

    with open(output_lst, 'w') as f:
        for img, lbl in zip(images, labels):
            # Generate full paths and normalize to use forward slashes
            img_path = os.path.join(image_dir, img).replace("\\", "/")
            lbl_path = os.path.join(label_dir, lbl).replace("\\", "/")
            # Write formatted line with consistent spacing
            f.write(f"{img_path} {lbl_path}\n")

# Paths to the LoveDA dataset directories
urban_train_image_dir = "data/loveda/train/Urban/images_png"
urban_train_label_dir = "data/loveda/train/Urban/masks_png"

urban_test_image_dir = "data/loveda/val/Urban/images_png"
urban_test_label_dir = "data/loveda/val/Urban/masks_png"

rural_train_image_dir = "data/loveda/train/Rural/images_png"
rural_train_label_dir = "data/loveda/train/Rural/masks_png"

rural_test_image_dir = "data/loveda/val/Rural/images_png"
rural_test_label_dir = "data/loveda/val/Rural/masks_png"



# train on urban - test on urban
train_lst_path = "data/list/loveda/urban_urban/train.lst"
test_lst_path = "data/list/loveda/urban_urban/val.lst"

# Create .lst files
create_lst_file(urban_train_image_dir, urban_train_label_dir, train_lst_path)
create_lst_file(urban_test_image_dir, urban_test_label_dir, test_lst_path)

train_lst_path = "data/list/loveda/urban_rural/train.lst"
target_lst_path = "data/list/loveda/rural/train.lst"
test_lst_path = "data/list/loveda/urban_rural/val.lst"

# Create .lst files
create_lst_file(urban_train_image_dir, urban_train_label_dir, train_lst_path)
create_lst_file(rural_train_image_dir, rural_train_label_dir, target_lst_path)
create_lst_file(rural_test_image_dir, rural_test_label_dir, test_lst_path)

In [10]:
# ------------------------------------------------------------------------------
# Copyright (c) Microsoft
# Licensed under the MIT License.
# Written by Ke Sun (sunk@mail.ustc.edu.cn)
# ------------------------------------------------------------------------------

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path as osp
import sys


def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)


# model_utils.py

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

BatchNorm2d = nn.BatchNorm2d
bn_mom = 0.1
algc = False

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, no_relu=False):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = BatchNorm2d(planes, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               padding=1, bias=False)
        self.bn2 = BatchNorm2d(planes, momentum=bn_mom)
        self.downsample = downsample
        self.stride = stride
        self.no_relu = no_relu

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        if self.no_relu:
            return out
        else:
            return self.relu(out)

class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None, no_relu=True):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm2d(planes, momentum=bn_mom)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = BatchNorm2d(planes, momentum=bn_mom)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3 = BatchNorm2d(planes * self.expansion, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.no_relu = no_relu

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        if self.no_relu:
            return out
        else:
            return self.relu(out)

class segmenthead(nn.Module):

    def __init__(self, inplanes, interplanes, outplanes, scale_factor=None):
        super(segmenthead, self).__init__()
        self.bn1 = BatchNorm2d(inplanes, momentum=bn_mom)
        self.conv1 = nn.Conv2d(inplanes, interplanes, kernel_size=3, padding=1, bias=False)
        self.bn2 = BatchNorm2d(interplanes, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(interplanes, outplanes, kernel_size=1, padding=0, bias=True)
        self.scale_factor = scale_factor

    def forward(self, x):

        x = self.conv1(self.relu(self.bn1(x)))
        out = self.conv2(self.relu(self.bn2(x)))

        if self.scale_factor is not None:
            height = x.shape[-2] * self.scale_factor
            width = x.shape[-1] * self.scale_factor
            out = F.interpolate(out,
                        size=[height, width],
                        mode='bilinear', align_corners=algc)

        return out

class DAPPM(nn.Module):
    def __init__(self, inplanes, branch_planes, outplanes, BatchNorm=nn.BatchNorm2d):
        super(DAPPM, self).__init__()
        bn_mom = 0.1
        self.scale1 = nn.Sequential(nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale2 = nn.Sequential(nn.AvgPool2d(kernel_size=9, stride=4, padding=4),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale3 = nn.Sequential(nn.AvgPool2d(kernel_size=17, stride=8, padding=8),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale4 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale0 = nn.Sequential(
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.process1 = nn.Sequential(
                                    BatchNorm(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process2 = nn.Sequential(
                                    BatchNorm(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process3 = nn.Sequential(
                                    BatchNorm(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process4 = nn.Sequential(
                                    BatchNorm(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.compression = nn.Sequential(
                                    BatchNorm(branch_planes * 5, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes * 5, outplanes, kernel_size=1, bias=False),
                                    )
        self.shortcut = nn.Sequential(
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=False),
                                    )

    def forward(self, x):
        width = x.shape[-1]
        height = x.shape[-2]
        x_list = []

        x_list.append(self.scale0(x))
        x_list.append(self.process1((F.interpolate(self.scale1(x),
                        size=[height, width],
                        mode='bilinear', align_corners=algc)+x_list[0])))
        x_list.append((self.process2((F.interpolate(self.scale2(x),
                        size=[height, width],
                        mode='bilinear', align_corners=algc)+x_list[1]))))
        x_list.append(self.process3((F.interpolate(self.scale3(x),
                        size=[height, width],
                        mode='bilinear', align_corners=algc)+x_list[2])))
        x_list.append(self.process4((F.interpolate(self.scale4(x),
                        size=[height, width],
                        mode='bilinear', align_corners=algc)+x_list[3])))

        out = self.compression(torch.cat(x_list, 1)) + self.shortcut(x)
        return out

class PAPPM(nn.Module):
    def __init__(self, inplanes, branch_planes, outplanes, BatchNorm=nn.BatchNorm2d):
        super(PAPPM, self).__init__()
        bn_mom = 0.1
        self.scale1 = nn.Sequential(nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale2 = nn.Sequential(nn.AvgPool2d(kernel_size=9, stride=4, padding=4),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale3 = nn.Sequential(nn.AvgPool2d(kernel_size=17, stride=8, padding=8),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale4 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )

        self.scale0 = nn.Sequential(
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )

        self.scale_process = nn.Sequential(
                                    BatchNorm(branch_planes*4, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes*4, branch_planes*4, kernel_size=3, padding=1, groups=4, bias=False),
                                    )


        self.compression = nn.Sequential(
                                    BatchNorm(branch_planes * 5, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes * 5, outplanes, kernel_size=1, bias=False),
                                    )

        self.shortcut = nn.Sequential(
                                    BatchNorm(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=False),
                                    )


    def forward(self, x):
        width = x.shape[-1]
        height = x.shape[-2]
        scale_list = []

        x_ = self.scale0(x)
        scale_list.append(F.interpolate(self.scale1(x), size=[height, width],
                        mode='bilinear', align_corners=algc)+x_)
        scale_list.append(F.interpolate(self.scale2(x), size=[height, width],
                        mode='bilinear', align_corners=algc)+x_)
        scale_list.append(F.interpolate(self.scale3(x), size=[height, width],
                        mode='bilinear', align_corners=algc)+x_)
        scale_list.append(F.interpolate(self.scale4(x), size=[height, width],
                        mode='bilinear', align_corners=algc)+x_)

        scale_out = self.scale_process(torch.cat(scale_list, 1))

        out = self.compression(torch.cat([x_,scale_out], 1)) + self.shortcut(x)
        return out


class PagFM(nn.Module):
    def __init__(self, in_channels, mid_channels, after_relu=False, with_channel=False, BatchNorm=nn.BatchNorm2d):
        super(PagFM, self).__init__()
        self.with_channel = with_channel
        self.after_relu = after_relu
        self.f_x = nn.Sequential(
                                nn.Conv2d(in_channels, mid_channels,
                                          kernel_size=1, bias=False),
                                BatchNorm(mid_channels)
                                )
        self.f_y = nn.Sequential(
                                nn.Conv2d(in_channels, mid_channels,
                                          kernel_size=1, bias=False),
                                BatchNorm(mid_channels)
                                )
        if with_channel:
            self.up = nn.Sequential(
                                    nn.Conv2d(mid_channels, in_channels,
                                              kernel_size=1, bias=False),
                                    BatchNorm(in_channels)
                                   )
        if after_relu:
            self.relu = nn.ReLU(inplace=True)

    def forward(self, x, y):
        input_size = x.size()
        if self.after_relu:
            y = self.relu(y)
            x = self.relu(x)

        y_q = self.f_y(y)
        y_q = F.interpolate(y_q, size=[input_size[2], input_size[3]],
                            mode='bilinear', align_corners=False)
        x_k = self.f_x(x)

        if self.with_channel:
            sim_map = torch.sigmoid(self.up(x_k * y_q))
        else:
            sim_map = torch.sigmoid(torch.sum(x_k * y_q, dim=1).unsqueeze(1))

        y = F.interpolate(y, size=[input_size[2], input_size[3]],
                            mode='bilinear', align_corners=False)
        x = (1-sim_map)*x + sim_map*y

        return x

class Light_Bag(nn.Module):
    def __init__(self, in_channels, out_channels, BatchNorm=nn.BatchNorm2d):
        super(Light_Bag, self).__init__()
        self.conv_p = nn.Sequential(
                                nn.Conv2d(in_channels, out_channels,
                                          kernel_size=1, bias=False),
                                BatchNorm(out_channels)
                                )
        self.conv_i = nn.Sequential(
                                nn.Conv2d(in_channels, out_channels,
                                          kernel_size=1, bias=False),
                                BatchNorm(out_channels)
                                )

    def forward(self, p, i, d):
        edge_att = torch.sigmoid(d)

        p_add = self.conv_p((1-edge_att)*i + p)
        i_add = self.conv_i(i + edge_att*p)

        return p_add + i_add


class DDFMv2(nn.Module):
    def __init__(self, in_channels, out_channels, BatchNorm=nn.BatchNorm2d):
        super(DDFMv2, self).__init__()
        self.conv_p = nn.Sequential(
                                BatchNorm(in_channels),
                                nn.ReLU(inplace=True),
                                nn.Conv2d(in_channels, out_channels,
                                          kernel_size=1, bias=False),
                                BatchNorm(out_channels)
                                )
        self.conv_i = nn.Sequential(
                                BatchNorm(in_channels),
                                nn.ReLU(inplace=True),
                                nn.Conv2d(in_channels, out_channels,
                                          kernel_size=1, bias=False),
                                BatchNorm(out_channels)
                                )

    def forward(self, p, i, d):
        edge_att = torch.sigmoid(d)

        p_add = self.conv_p((1-edge_att)*i + p)
        i_add = self.conv_i(i + edge_att*p)

        return p_add + i_add

class Bag(nn.Module):
    def __init__(self, in_channels, out_channels, BatchNorm=nn.BatchNorm2d):
        super(Bag, self).__init__()

        self.conv = nn.Sequential(
                                BatchNorm(in_channels),
                                nn.ReLU(inplace=True),
                                nn.Conv2d(in_channels, out_channels,
                                          kernel_size=3, padding=1, bias=False)
                                )


    def forward(self, p, i, d):
        edge_att = torch.sigmoid(d)
        return self.conv(edge_att*p + (1-edge_att)*i)


# pidnet.py

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import logging

BatchNorm2d = nn.BatchNorm2d
bn_mom = 0.1
algc = False

class PIDNet(nn.Module):

    def __init__(self, m=2, n=3, num_classes=19, planes=64, ppm_planes=96, head_planes=128, augment=True):
        super(PIDNet, self).__init__()
        self.augment = augment

        # I Branch
        self.conv1 =  nn.Sequential(
                          nn.Conv2d(3,planes,kernel_size=3, stride=2, padding=1),
                          BatchNorm2d(planes, momentum=bn_mom),
                          nn.ReLU(inplace=True),
                          nn.Conv2d(planes,planes,kernel_size=3, stride=2, padding=1),
                          BatchNorm2d(planes, momentum=bn_mom),
                          nn.ReLU(inplace=True),
                      )

        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(BasicBlock, planes, planes, m)
        self.layer2 = self._make_layer(BasicBlock, planes, planes * 2, m, stride=2)
        self.layer3 = self._make_layer(BasicBlock, planes * 2, planes * 4, n, stride=2)
        self.layer4 = self._make_layer(BasicBlock, planes * 4, planes * 8, n, stride=2)
        self.layer5 =  self._make_layer(Bottleneck, planes * 8, planes * 8, 2, stride=2)

        # P Branch
        self.compression3 = nn.Sequential(
                                          nn.Conv2d(planes * 4, planes * 2, kernel_size=1, bias=False),
                                          BatchNorm2d(planes * 2, momentum=bn_mom),
                                          )

        self.compression4 = nn.Sequential(
                                          nn.Conv2d(planes * 8, planes * 2, kernel_size=1, bias=False),
                                          BatchNorm2d(planes * 2, momentum=bn_mom),
                                          )
        self.pag3 = PagFM(planes * 2, planes)
        self.pag4 = PagFM(planes * 2, planes)

        self.layer3_ = self._make_layer(BasicBlock, planes * 2, planes * 2, m)
        self.layer4_ = self._make_layer(BasicBlock, planes * 2, planes * 2, m)
        self.layer5_ = self._make_layer(Bottleneck, planes * 2, planes * 2, 1)

        # D Branch
        if m == 2:
            self.layer3_d = self._make_single_layer(BasicBlock, planes * 2, planes)
            self.layer4_d = self._make_layer(Bottleneck, planes, planes, 1)
            self.diff3 = nn.Sequential(
                                        nn.Conv2d(planes * 4, planes, kernel_size=3, padding=1, bias=False),
                                        BatchNorm2d(planes, momentum=bn_mom),
                                        )
            self.diff4 = nn.Sequential(
                                     nn.Conv2d(planes * 8, planes * 2, kernel_size=3, padding=1, bias=False),
                                     BatchNorm2d(planes * 2, momentum=bn_mom),
                                     )
            self.spp = PAPPM(planes * 16, ppm_planes, planes * 4)
            self.dfm = Light_Bag(planes * 4, planes * 4)
        else:
            self.layer3_d = self._make_single_layer(BasicBlock, planes * 2, planes * 2)
            self.layer4_d = self._make_single_layer(BasicBlock, planes * 2, planes * 2)
            self.diff3 = nn.Sequential(
                                        nn.Conv2d(planes * 4, planes * 2, kernel_size=3, padding=1, bias=False),
                                        BatchNorm2d(planes * 2, momentum=bn_mom),
                                        )
            self.diff4 = nn.Sequential(
                                     nn.Conv2d(planes * 8, planes * 2, kernel_size=3, padding=1, bias=False),
                                     BatchNorm2d(planes * 2, momentum=bn_mom),
                                     )
            self.spp = DAPPM(planes * 16, ppm_planes, planes * 4)
            self.dfm = Bag(planes * 4, planes * 4)

        self.layer5_d = self._make_layer(Bottleneck, planes * 2, planes * 2, 1)

        # Prediction Head
        if self.augment:
            self.seghead_p = segmenthead(planes * 2, head_planes, num_classes)
            self.seghead_d = segmenthead(planes * 2, planes, 1)

        self.final_layer = segmenthead(planes * 4, head_planes, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def _make_layer(self, block, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=bn_mom),
            )

        layers = []
        layers.append(block(inplanes, planes, stride, downsample))
        inplanes = planes * block.expansion
        for i in range(1, blocks):
            if i == (blocks-1):
                layers.append(block(inplanes, planes, stride=1, no_relu=True))
            else:
                layers.append(block(inplanes, planes, stride=1, no_relu=False))

        return nn.Sequential(*layers)

    def _make_single_layer(self, block, inplanes, planes, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=bn_mom),
            )

        layer = block(inplanes, planes, stride, downsample, no_relu=True)

        return layer

    def forward(self, x):

        width_output = x.shape[-1] // 8
        height_output = x.shape[-2] // 8

        x = self.conv1(x)
        x = self.layer1(x)
        x = self.relu(self.layer2(self.relu(x)))
        x_ = self.layer3_(x)
        x_d = self.layer3_d(x)

        x = self.relu(self.layer3(x))
        x_ = self.pag3(x_, self.compression3(x))
        x_d = x_d + F.interpolate(
                        self.diff3(x),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=algc)
        if self.augment:
            temp_p = x_

        x = self.relu(self.layer4(x))
        x_ = self.layer4_(self.relu(x_))
        x_d = self.layer4_d(self.relu(x_d))

        x_ = self.pag4(x_, self.compression4(x))
        x_d = x_d + F.interpolate(
                        self.diff4(x),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=algc)
        if self.augment:
            temp_d = x_d

        x_ = self.layer5_(self.relu(x_))
        x_d = self.layer5_d(self.relu(x_d))
        x = F.interpolate(
                        self.spp(self.layer5(x)),
                        size=[height_output, width_output],
                        mode='bilinear', align_corners=algc)

        x_ = self.final_layer(self.dfm(x_, x, x_d))

        if self.augment:
            x_extra_p = self.seghead_p(temp_p)
            x_extra_d = self.seghead_d(temp_d)
            return [x_extra_p, x_, x_extra_d]
        else:
            return x_

def get_seg_model(cfg, imgnet_pretrained):

    if 's' in cfg.MODEL.NAME:
        model = PIDNet(m=2, n=3, num_classes=cfg.DATASET.NUM_CLASSES, planes=32, ppm_planes=96, head_planes=128, augment=True)
    elif 'm' in cfg.MODEL.NAME:
        model = PIDNet(m=2, n=3, num_classes=cfg.DATASET.NUM_CLASSES, planes=64, ppm_planes=96, head_planes=128, augment=True)
    else:
        model = PIDNet(m=3, n=4, num_classes=cfg.DATASET.NUM_CLASSES, planes=64, ppm_planes=112, head_planes=256, augment=True)

    if imgnet_pretrained:
        pretrained_state = torch.load(cfg.MODEL.PRETRAINED, map_location='cpu')['state_dict']
        model_dict = model.state_dict()
        pretrained_state = {k: v for k, v in pretrained_state.items() if (k in model_dict and v.shape == model_dict[k].shape)}
        model_dict.update(pretrained_state)
        msg = 'Loaded {} parameters!'.format(len(pretrained_state))
        logging.info('Attention!!!')
        logging.info(msg)
        logging.info('Over!!!')
        model.load_state_dict(model_dict, strict = False)
    else:
        pretrained_dict = torch.load(cfg.MODEL.PRETRAINED, map_location='cpu')
        if 'state_dict' in pretrained_dict:
            pretrained_dict = pretrained_dict['state_dict']
        model_dict = model.state_dict()
        pretrained_dict = {k[6:]: v for k, v in pretrained_dict.items() if (k[6:] in model_dict and v.shape == model_dict[k[6:]].shape)}
        msg = 'Loaded {} parameters!'.format(len(pretrained_dict))
        logging.info('Attention!!!')
        logging.info(msg)
        logging.info('Over!!!')
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict, strict = False)

    return model

def get_pred_model(name, num_classes):

    if 's' in name:
        model = PIDNet(m=2, n=3, num_classes=num_classes, planes=32, ppm_planes=96, head_planes=128, augment=False)
    elif 'm' in name:
        model = PIDNet(m=2, n=3, num_classes=num_classes, planes=64, ppm_planes=96, head_planes=128, augment=False)
    else:
        model = PIDNet(m=3, n=4, num_classes=num_classes, planes=64, ppm_planes=112, head_planes=256, augment=False)

    return model

# base_dataset.py

In [13]:
import cv2
import numpy as np
import random
import albumentations as A
from torch.nn import functional as F
from torch.utils import data
import matplotlib.pyplot as plt
y_k_size = 6
x_k_size = 6

def show_images(x_original, x_augmented, unnormalize = False):

    if unnormalize:
        # ImageNet mean and std
        imagenet_mean = np.array([0.485, 0.456, 0.406])[:, None, None]
        imagenet_std = np.array([0.229, 0.224, 0.225])[:, None, None]

        # Denormalize using NumPy broadcasting
        x_original = x_original * imagenet_std + imagenet_mean
        x_augmented = x_augmented * imagenet_std + imagenet_mean

        # Clip to [0, 1] in case of overflows
        x_original = np.clip(x_original, 0, 1)
        x_augmented = np.clip(x_augmented, 0, 1)

        # Transpose to HWC for matplotlib
        x_original = np.transpose(x_original, (1, 2, 0))
        x_augmented = np.transpose(x_augmented, (1, 2, 0))

    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(x_original)
    axs[0].set_title("Original Image")
    axs[0].axis("off")

    axs[1].imshow(x_augmented)
    axs[1].set_title("Augmented Image")
    axs[1].axis("off")

    plt.tight_layout()
    plt.show()

class BaseDataset(data.Dataset):
    def __init__(self,
                 ignore_label=255,
                 base_size=2048,
                 crop_size=(1024, 1024),
                 scale_factor=16,
                 mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]):

        self.base_size = base_size
        self.crop_size = crop_size
        self.ignore_label = ignore_label

        self.mean = mean
        self.std = std
        self.scale_factor = scale_factor

        self.files = []

    def __len__(self):
        return len(self.files)

    def input_transform(self, image, city=False):
        if city:
            image = image.astype(np.float32)[:, :, ::-1]
        else:
            image = image.astype(np.float32)
        image = image / 255.0
        image -= self.mean
        image /= self.std
        return image

    def label_transform(self, label):
        return np.array(label).astype(np.uint8)

    def pad_image(self, image, h, w, size, padvalue):
        pad_h = max(size[0] - h, 0)
        pad_w = max(size[1] - w, 0)

        # Se non è necessario il padding, restituisci l'immagine originale
        if pad_h == 0 and pad_w == 0:
            return image

        # Verifica il formato dell'immagine (deve essere H, W, C)
        if len(image.shape) == 3 and image.shape[0] <= 3:  # Se è in formato (C, H, W)
            image = np.transpose(image, (1, 2, 0))  # Converti in (H, W, C)

        # Aggiungi il padding
        pad_image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=padvalue)

        # Ripristina il formato originale (C, H, W) se necessario
        if len(image.shape) == 3 and image.shape[2] <= 3:  # Se era in formato (C, H, W)
            pad_image = np.transpose(pad_image, (2, 0, 1))  # Converti di nuovo in (C, H, W)

        return pad_image

    def rand_crop(self, image, label, edge):
        # Verifica il formato dell'immagine
        if len(image.shape) == 3 and image.shape[0] <= 3:  # Se è in formato (C, H, W)
            image = np.transpose(image, (1, 2, 0))  # Converti in (H, W, C)

        h, w = image.shape[:2]

        # Aggiungi padding se necessario
        if h < self.crop_size[0] or w < self.crop_size[1]:
            image = self.pad_image(image, h, w, self.crop_size, (0.0, 0.0, 0.0))
            label = self.pad_image(label, h, w, self.crop_size, (self.ignore_label,))
            edge = self.pad_image(edge, h, w, self.crop_size, (0.0,))

        # Aggiorna le dimensioni dopo il padding
        new_h, new_w = label.shape
        if new_h < self.crop_size[0] or new_w < self.crop_size[1]:
            raise ValueError(f"Dimensioni insufficienti per il ritaglio: label={label.shape}, crop_size={self.crop_size}")

        # Calcola le coordinate per il ritaglio casuale
        x = random.randint(0, new_w - self.crop_size[1])
        y = random.randint(0, new_h - self.crop_size[0])

        # Esegui il ritaglio
        image = image[y:y+self.crop_size[0], x:x+self.crop_size[1]]
        label = label[y:y+self.crop_size[0], x:x+self.crop_size[1]]
        edge = edge[y:y+self.crop_size[0], x:x+self.crop_size[1]]

        #in questo modo l'iimagine è 512x512x3
        #se volessi croppare quella regione
        '''
        # Estrai la regione da sfocare
        cropped_region = image[y:y+crop_size[0], x:x+crop_size[1]]

        # Applica il Gaussian Blur alla regione
        blurred_region = cv2.GaussianBlur(cropped_region, (15, 15), 0)

        # Sostituisci la regione originale con quella sfocata
        augmented_image = image.copy()
        augmented_image[y:y+crop_size[0], x:x+crop_size[1]] = blurred_region
        '''

        return image, label, edge

    def multi_scale_aug(self, image, label=None, edge=None,
                        rand_scale=1, rand_crop=True):
        long_size = int(self.base_size * rand_scale + 0.5)
        h, w = image.shape[:2]
        if h > w:
            new_h = long_size
            new_w = int(w * long_size / h + 0.5)
        else:
            new_w = long_size
            new_h = int(h * long_size / w + 0.5)

        image = cv2.resize(image, (new_w, new_h),
                           interpolation=cv2.INTER_LINEAR)
        if label is not None:
            label = cv2.resize(label, (new_w, new_h),
                               interpolation=cv2.INTER_NEAREST)
            if edge is not None:
                edge = cv2.resize(edge, (new_w, new_h),
                                   interpolation=cv2.INTER_NEAREST)
        else:
            return image

        if rand_crop:
            image, label, edge = self.rand_crop(image, label, edge)

        return image, label, edge


    def gen_sample(self, image, label, edge_pad=True, edge_size=4, city=False, transform=None, show=False):


        if transform:
            # Pass both image and mask
            augmented = transform(image=image, mask=label)

            if show:
                show_images(image, augmented["image"])

            # Extract results
            image = augmented['image']
            label = augmented['mask']



        #It' important keeping the edge generation after the data augmentation
        edge = cv2.Canny(label, 0.1, 0.2)
        kernel = np.ones((edge_size, edge_size), np.uint8)
        if edge_pad:
            edge = edge[y_k_size:-y_k_size, x_k_size:-x_k_size]
            edge = np.pad(edge, ((y_k_size,y_k_size),(x_k_size,x_k_size)), mode='constant')
        edge = (cv2.dilate(edge, kernel, iterations=1)>50)*1.0


        #trasformazioni di input
        image = self.input_transform(image, city=city) #Se city=True, converte l'immagine da RGB in BGR per opencv
        label = self.label_transform(label) #converte la label in un array di interi
        image = image.transpose((2, 0, 1)) #H,W,C -> C,H,W

        return image, label, edge


    def inference(self, config, model, image):
        size = image.size()
        pred = model(image)

        if config.MODEL.NUM_OUTPUTS > 1:
            pred = pred[config.TEST.OUTPUT_INDEX]


        pred = F.interpolate(
            input=pred, size=size[-2:],
            mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS
        )


        return pred.exp()

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# discriminator.py

In [14]:
#------------------------------------------------------------------------
    #Discriminator based on GAN

import torch.nn as nn
import torch.nn.functional as F


class FCDiscriminator(nn.Module):

    def __init__(self, num_classes, ndf = 64):
        super(FCDiscriminator, self).__init__()

        self.conv1 = nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(ndf, ndf*2, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(ndf*2, ndf*4, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(ndf*4, ndf*8, kernel_size=4, stride=2, padding=1)
        self.classifier = nn.Conv2d(ndf*8, 1, kernel_size=4, stride=2, padding=1)

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        #self.up_sample = nn.Upsample(scale_factor=32, mode='bilinear')
        #self.sigmoid = nn.Sigmoid() #non uso sigmoid per la stabilità numerica a discapito di avere i logits e non le probabilità


    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.conv4(x)
        x = self.leaky_relu(x)
        x = self.classifier(x)
        #x = self.up_sample(x)
        #x = self.sigmoid(x)

        return x


# loveda.py

In [ ]:
import cv2
import os
import numpy as np
import torch
import random
import logging
from PIL import Image
import torchvision.transforms as tf
import matplotlib.pyplot as plt

def compare_images(image, blurred_image):
    # Compute the absolute difference between the images
    # Ensure both images are in the same format (H, W, C)
    blurred_image = blurred_image.transpose(1, 2, 0)  # Change (C, H, W) to (H, W, C)

    # Compute the absolute difference between the images
    diff = np.abs(image - blurred_image)

    # Plot the images and their difference side by side
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    axs[0].imshow(image)  # Original image
    axs[0].set_title("Original Image")
    axs[0].axis("off")

    axs[1].imshow(blurred_image)  # Blurred image
    axs[1].set_title("Blurred Image")
    axs[1].axis("off")

    axs[2].imshow(diff)  # Difference image
    axs[2].set_title("Difference Image")
    axs[2].axis("off")

    plt.show()

    return diff


def show_images(x_original, x_augmented, unnormalize = False):

    if unnormalize:
        # ImageNet mean and std
        imagenet_mean = np.array([0.485, 0.456, 0.406])[:, None, None]
        imagenet_std = np.array([0.229, 0.224, 0.225])[:, None, None]

        # Denormalize using NumPy broadcasting
        x_original = x_original * imagenet_std + imagenet_mean
        x_augmented = x_augmented * imagenet_std + imagenet_mean

        # Clip to [0, 1] in case of overflows
        x_original = np.clip(x_original, 0, 1)
        x_augmented = np.clip(x_augmented, 0, 1)

        # Transpose to HWC for matplotlib
        x_original = np.transpose(x_original, (1, 2, 0))
        x_augmented = np.transpose(x_augmented, (1, 2, 0))

    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(x_original)
    axs[0].set_title("Original Image")
    axs[0].axis("off")

    axs[1].imshow(x_augmented)
    axs[1].set_title("Augmented Image")
    axs[1].axis("off")

    plt.tight_layout()
    plt.show()


#classe per fare la data augmentation
class DataAugmentation:
    def __init__(self, config, dataset_instance):
        self.enable = config["ENABLE"]
        self.probability = config["PROBABILITY"]
        self.techniques = config["TECHNIQUES"]
        self.dataset = dataset_instance  # Riferimento all'istanza del dataset

    def apply(self, image, label, edge):

        if not self.enable or random.random() > self.probability: #50% di probabilità di applicare la data augmentation
            return image,label,edge #non faccio augmentation

        if self.techniques.get("HORIZONTAL_FLIP", False):
            image,label,edge = self.horizontal_flip(image, label, edge)

        if self.techniques.get("GAUSSIAN_BLUR", False):
            image, label, edge = self.gaussian_blur(image, label, edge)

        if self.techniques.get("MULTIPLY", False):
            image, label, edge = self.multiply(image, label, edge)

        if self.techniques.get("RANDOM_BRIGHTNESS", False):
            image, label, edge = self.random_brightness(image, label, edge)

        if self.techniques.get("RANDOM_CROP", False):
            image, label, edge = self.random_crop(image, label, edge)


        return image, label, edge



    def random_crop(self, image, label, edge):
        return self.dataset.rand_crop(image, label, edge)  # Usa l'istanza del dataset



    def horizontal_flip(self, image, label, edge):
        # Inverti orizzontalmente immagine, label ed edge
        flipped_image = image[:, :, ::-1]
        flipped_label = label[:, ::-1]
        flipped_edge = edge[:, ::-1]
        return flipped_image, flipped_label, flipped_edge



    def gaussian_blur(self, image, label, edge, kernel_size=5, show = False):
        # Applica il Gaussian Blur solo all'immagine
        transposed_image = image.transpose(1, 2, 0)  # From (C, H, W) to (H, W, C)

        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(transposed_image, (kernel_size, kernel_size), 0)

        # If you want to return it to the PyTorch format (C, H, W)
        blurred_image = blurred_image.transpose(2, 0, 1)  # From (H, W, C) to (C, H, W)

        if show:
            show_images(image, blurred_image)

        return blurred_image, label, edge



    def multiply(self, image, label, edge, factor_range=(0.8, 1.2), show = False):
        # Convert image to float32 to avoid overflow issues
        factor = random.uniform(*factor_range)
        image = image.astype(np.float32)  # Ensure safe multiplication

        # Check if image is normalized (0-1), rescale before multiplication
        if image.max() <= 1.0:
            image *= 255.0  # Scale to 0-255 range before multiplication

        multiplied_image = image * factor

        if show:
            show_images(image, multiplied_image)

        return multiplied_image, label, edge

    def random_brightness(self, image, label, edge, brightness_range=(-0.5, 0.5), show = False):
        # Modify image brightness
        brightness = np.float32(np.random.uniform(*brightness_range))
        brightened_image = image + brightness  # Keep within [0,1]

        if show:
            show_images(image, brightened_image)
        return brightened_image, label, edge



class LoveDA(BaseDataset):
    def __init__(self,
                 root,
                 list_path,
                 num_classes=7,
                 multi_scale=False,
                 flip=False,
                 ignore_label=0,
                 base_size=1024,
                 crop_size=(512, 512),
                 scale_factor=16,
                 enable_augmentation=False,
                 augmentation_probability=0.5,
                 horizontal_flip=False,
                 gaussian_blur=False,
                 multiply=False,
                 random_brightness=False,
                 random_crop=False,
                 mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225],
                 bd_dilate_size=4,
                 pseudo_label=False,
                 transform=None):

        # estende il base_dataset
        super(LoveDA, self).__init__(ignore_label, base_size,
                                     crop_size, scale_factor, mean, std)

        self.root = root
        self.list_path = list_path
        self.num_classes = num_classes
        self.multi_scale = multi_scale
        self.flip = flip
        self.ignore_label = ignore_label
        self.base_size = base_size
        self.crop_size = crop_size
        self.scale_factor = scale_factor
        self.enable_augmentation = enable_augmentation
        self.augmentation_probability = augmentation_probability
        self.horizontal_flip = horizontal_flip
        self.gaussian_blur = gaussian_blur
        self.multiply = multiply
        self.random_brightness = random_brightness
        self.random_crop = random_crop
        self.bd_dilate_size = bd_dilate_size

        self.img_list = [line.strip().split() for line in open(root + list_path)]
        self.files = self.read_files()
        self.color_list = [[0, 0, 0], [1, 1, 1], [2, 2, 2],
                            [3, 3, 3], [4, 4, 4], [5, 5, 5], [6, 6, 6], [7, 7, 7]]
        self.class_weights = torch.tensor([0.000000, 0.116411, 0.266041, 0.607794, 1.511413, 0.745507, 0.712438, 3.040396])
        self.pseudo_label = pseudo_label
        self.transform=transform

    def read_files(self):
        files = []

        for item in self.img_list:
            image_path, label_path = item
            name = os.path.splitext(os.path.basename(label_path))[0]
            files.append({
                "img": image_path,
                "label": label_path,
                "name": name
            })

        return files

    # da immagine a label
    def color2label(self, color_map):
        label = np.ones(color_map.shape[:2]) * self.ignore_label
        for i, v in enumerate(self.color_list):
            label[(color_map == v).sum(2) == 3] = i

        return label.astype(np.uint8)

    def convert_label(self, label, inverse=False):
        temp = label.copy()
        if inverse:
            for v, k in self.label_mapping.items():
                label[temp == k] = v
        else:
            for k, v in self.label_mapping.items():
                label[temp == k] = v
        return label

    # da label a immagine
    def label2color(self, label):
        color_map = np.zeros(label.shape + (3,))
        for i, v in enumerate(self.color_list):
            color_map[label == i] = self.color_list[i]

        return color_map.astype(np.uint8)

    def __getitem__(self, index):
        item = self.files[index]
        name = item["name"]
        image = cv2.imread(item["img"], cv2.IMREAD_COLOR)

        size = image.shape

        label = cv2.imread(item["label"], cv2.IMREAD_GRAYSCALE)



        #edge (H,W)
        image, label, edge = self.gen_sample(image, label, edge_pad=False,
                                             edge_size=self.bd_dilate_size, city=False, transform=self.transform, show=False) #image diventa (C,H,W)

        return image.copy(), label.copy(), edge.copy(), np.array(size), name

    def single_scale_inference(self, config, model, image):
        pred = self.inference(config, model, image)
        return pred

    def save_pred(self, preds, sv_path, name):
        preds = np.asarray(np.argmax(preds.cpu(), axis=1), dtype=np.uint8)
        for i in range(preds.shape[0]):
            pred = self.label2color(preds[i])
            save_img = Image.fromarray(pred)
            save_img.save(os.path.join(sv_path, name[i] + '.png'))

# criterion.py

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from configs import config


class CrossEntropy(nn.Module):
    def __init__(self, ignore_label=-1, weight=None):
        super(CrossEntropy, self).__init__()
        self.ignore_label = ignore_label
        self.criterion = nn.CrossEntropyLoss(
            weight=weight,
            ignore_index=ignore_label
        )

    def _forward(self, score, target):

        loss = self.criterion(score, target)

        return loss

    def forward(self, score, target):

        if config.MODEL.NUM_OUTPUTS == 1:
            score = [score]

        balance_weights = config.LOSS.BALANCE_WEIGHTS
        sb_weights = config.LOSS.SB_WEIGHTS
        if len(balance_weights) == len(score):
            return sum([w * self._forward(x, target) for (w, x) in zip(balance_weights, score)])
        elif len(score) == 1:
            return sb_weights * self._forward(score[0], target)

        else:
            raise ValueError("lengths of prediction and target are not identical!")




class OhemCrossEntropy(nn.Module):
    def __init__(self, ignore_label=-1, thres=0.7,
                 min_kept=100000, weight=None):
        super(OhemCrossEntropy, self).__init__()
        self.thresh = thres
        self.min_kept = max(1, min_kept)
        self.ignore_label = ignore_label
        self.criterion = nn.CrossEntropyLoss(
            weight=weight,
            ignore_index=ignore_label,
            reduction='none'
        )

    def _ce_forward(self, score, target):


        loss = self.criterion(score, target)

        return loss

    def _ohem_forward(self, score, target, **kwargs):

        pred = F.softmax(score, dim=1)
        pixel_losses = self.criterion(score, target).contiguous().view(-1)
        mask = target.contiguous().view(-1) != self.ignore_label

        tmp_target = target.clone()
        tmp_target[tmp_target == self.ignore_label] = 0
        pred = pred.gather(1, tmp_target.unsqueeze(1))
        pred, ind = pred.contiguous().view(-1,)[mask].contiguous().sort()
        min_value = pred[min(self.min_kept, pred.numel() - 1)]
        threshold = max(min_value, self.thresh)

        pixel_losses = pixel_losses[mask][ind]
        pixel_losses = pixel_losses[pred < threshold]
        return pixel_losses.mean()

    def forward(self, score, target):

        if not (isinstance(score, list) or isinstance(score, tuple)):
            score = [score]

        balance_weights = config.LOSS.BALANCE_WEIGHTS
        sb_weights = config.LOSS.SB_WEIGHTS
        if len(balance_weights) == len(score):
            functions = [self._ce_forward] * \
                (len(balance_weights) - 1) + [self._ohem_forward]
            return sum([
                w * func(x, target)
                for (w, x, func) in zip(balance_weights, score, functions)
            ])

        elif len(score) == 1:
            return sb_weights * self._ohem_forward(score[0], target)

        else:
            raise ValueError("lengths of prediction and target are not identical!")


def weighted_bce(bd_pre, target):
    n, c, h, w = bd_pre.size()
    log_p = bd_pre.permute(0,2,3,1).contiguous().view(1, -1)
    target_t = target.view(1, -1)

    pos_index = (target_t == 1)
    neg_index = (target_t == 0)

    weight = torch.zeros_like(log_p)
    pos_num = pos_index.sum()
    neg_num = neg_index.sum()
    sum_num = pos_num + neg_num
    weight[pos_index] = neg_num * 1.0 / sum_num
    weight[neg_index] = pos_num * 1.0 / sum_num

    loss = F.binary_cross_entropy_with_logits(log_p, target_t, weight, reduction='mean')

    return loss


class BondaryLoss(nn.Module):
    def __init__(self, coeff_bce = 20.0):
        super(BondaryLoss, self).__init__()
        self.coeff_bce = coeff_bce

    def forward(self, bd_pre, bd_gt):

        bce_loss = self.coeff_bce * weighted_bce(bd_pre, bd_gt)
        loss = bce_loss

        return loss

# utils.py

In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import logging
import time
from pathlib import Path

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from configs import default
config = default._C.clone()
update_config = default.update_config

class FullModel(nn.Module):

  def __init__(self, model, sem_loss, bd_loss):
    super(FullModel, self).__init__()
    self.model = model
    self.sem_loss = sem_loss
    self.bd_loss = bd_loss

  def pixel_acc(self, pred, label):
    _, preds = torch.max(pred, dim=1)
    valid = (label >= 0).long()
    acc_sum = torch.sum(valid * (preds == label).long())
    pixel_sum = torch.sum(valid)
    acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
    return acc

  def forward(self, inputs, labels, bd_gt, *args, **kwargs):

    outputs = self.model(inputs, *args, **kwargs)

    h, w = labels.size(1), labels.size(2)
    ph, pw = outputs[0].size(2), outputs[0].size(3)
    if ph != h or pw != w:
        for i in range(len(outputs)):
            outputs[i] = F.interpolate(outputs[i], size=(
                h, w), mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS)

    acc  = self.pixel_acc(outputs[-2], labels)
    loss_s = self.sem_loss(outputs[:-1], labels)
    loss_b = self.bd_loss(outputs[-1], bd_gt)

    filler = torch.ones_like(labels) * config.TRAIN.IGNORE_LABEL
    try:
        bd_label = torch.where(torch.sigmoid(outputs[-1][:, 0, :, :]) > 0.8, labels, filler) # 0.7
        loss_sb = self.sem_loss([outputs[-2]], bd_label)
    except:
        print("Error in loss computation")
        loss_sb = self.sem_loss([outputs[-2]], labels)
    loss = loss_s + loss_b + loss_sb

    return torch.unsqueeze(loss,0), outputs[:-1], acc, [loss_s, loss_b] #aoutputs[:-1] è una lista di tensori

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.initialized = False
        self.val = None
        self.avg = None
        self.sum = None
        self.count = None

    def initialize(self, val, weight):
        self.val = val
        self.avg = val
        self.sum = val * weight
        self.count = weight
        self.initialized = True

    def update(self, val, weight=1):
        if not self.initialized:
            self.initialize(val, weight)
        else:
            self.add(val, weight)

    def add(self, val, weight):
        self.val = val
        self.sum += val * weight
        self.count += weight
        self.avg = self.sum / self.count

    def value(self):
        return self.val

    def average(self):
        return self.avg

def create_logger(cfg, cfg_name, phase='train'):
    root_output_dir = Path(cfg.OUTPUT_DIR)

    if (
        not cfg.TRAIN.AUGMENTATION.TECHNIQUES.HORIZONTAL_FLIP and
        not cfg.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_BLUR and
        not cfg.TRAIN.AUGMENTATION.TECHNIQUES.RANDOM_CROP and
        not cfg.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_NOISE and
        not cfg.TRAIN.AUGMENTATION.TECHNIQUES.COLOR_JITTER
        ):
        folder_name = "no_aug"
    else:
        folder_name = "aug"

    if cfg.TRAIN.DACS.ENABLE:
        folder_name = "dacs"

    if cfg.TRAIN.GAN.ENABLE:
        folder_name = "gan"

    if cfg.TRAIN.AUGMENTATION.ENABLE:
        folder_name+= "_hf" if cfg.TRAIN.AUGMENTATION.TECHNIQUES.HORIZONTAL_FLIP else ""
        folder_name+= "_gb" if cfg.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_BLUR else ""
        folder_name+= "_rc" if cfg.TRAIN.AUGMENTATION.TECHNIQUES.RANDOM_CROP else ""
        folder_name+= "_cj" if cfg.TRAIN.AUGMENTATION.TECHNIQUES.COLOR_JITTER else ""
        folder_name+= "_gn" if cfg.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_NOISE else ""

    # set up logger
    if not root_output_dir.exists():
        print('=> creating {}'.format(root_output_dir))
        root_output_dir.mkdir()

    dataset = cfg.DATASET.DATASET
    model = cfg.MODEL.NAME
    cfg_name = os.path.basename(cfg_name).split('.')[0]

    final_output_dir = root_output_dir / dataset / cfg_name / folder_name

    print('=> creating {}'.format(final_output_dir))
    final_output_dir.mkdir(parents=True, exist_ok=True)

    time_str = time.strftime('%Y-%m-%d-%H-%M')
    log_file = '{}_{}_{}.log'.format(cfg_name, time_str, phase)
    final_log_file = final_output_dir / log_file
    head = '%(asctime)-15s %(message)s'
    logging.basicConfig(filename=str(final_log_file),
                        format=head)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    console = logging.StreamHandler()
    logging.getLogger('').addHandler(console)

    tensorboard_log_dir = Path(cfg.LOG_DIR) / dataset / model / \
            (cfg_name + '_' + time_str)
    print('=> creating {}'.format(tensorboard_log_dir))
    tensorboard_log_dir.mkdir(parents=True, exist_ok=True)

    return logger, str(final_output_dir), str(tensorboard_log_dir)

def get_confusion_matrix(label, pred, size, num_class, ignore=-1):
    """
    Calcute the confusion matrix by given label and pred
    """
    output = pred.cpu().numpy().transpose(0, 2, 3, 1)
    seg_pred = np.asarray(np.argmax(output, axis=3), dtype=np.uint8)
    seg_gt = np.asarray(
    label.cpu().numpy()[:, :size[-2], :size[-1]], dtype=int)

    ignore_index = seg_gt != ignore
    seg_gt = seg_gt[ignore_index]
    seg_pred = seg_pred[ignore_index]

    index = (seg_gt * num_class + seg_pred).astype('int32')
    label_count = np.bincount(index)
    confusion_matrix = np.zeros((num_class, num_class))

    for i_label in range(num_class):
        for i_pred in range(num_class):
            cur_index = i_label * num_class + i_pred
            if cur_index < len(label_count):
                confusion_matrix[i_label,
                                 i_pred] = label_count[cur_index]
    return confusion_matrix

def adjust_learning_rate(optimizer, base_lr, max_iters,cur_iters, power=0.9, nbb_mult=10):
    
    lr = base_lr*((1-float(cur_iters)/max_iters)**(power))
    optimizer.param_groups[0]['lr'] = lr
    if len(optimizer.param_groups) == 2:
        optimizer.param_groups[1]['lr'] = lr * nbb_mult
    return lr

# function.py

In [ ]:
import logging
import os
import time

import numpy as np
from tqdm import tqdm

import torch
from torch.nn import functional as F
import torch.nn as nn

def train(config, epoch, num_epoch, epoch_iters, base_lr,
          num_iters, trainloader, optimizer, model, writer_dict, targetloader=None):
    # Training
    model.train()

    batch_time = AverageMeter()
    ave_loss = AverageMeter()
    ave_acc  = AverageMeter()
    avg_sem_loss = AverageMeter()
    avg_bce_loss = AverageMeter()
    tic = time.time()
    cur_iters = epoch*epoch_iters
    writer = writer_dict['writer']
    global_steps = writer_dict['train_global_steps']

    if targetloader is not None:
        target_iter = iter(targetloader)

    for i_iter, batch in enumerate(trainloader, 0):
       
        images, labels, bd_gts, _, _ = batch
        images = images.cuda()
        labels = labels.long().cuda()
        bd_gts = bd_gts.float().cuda()


        losses, _, acc, loss_list = model(images, labels, bd_gts)
        loss = losses.mean()
        acc  = acc.mean()
        sem_loss = loss_list[0]
        bce_loss = loss_list[1]

        model.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - tic)
        tic = time.time()

        # update average loss
        ave_loss.update(loss.item())
        ave_acc.update(acc.item())
        avg_sem_loss.update(sem_loss.mean().item())
        avg_bce_loss.update(bce_loss.mean().item())

        lr = adjust_learning_rate(optimizer,
                                  base_lr,
                                  num_iters,
                                  i_iter+cur_iters)

        if i_iter % config.PRINT_FREQ == 0:
            msg = 'Epoch: [{}/{}] Iter:[{}/{}], Time: {:.2f}, ' \
                  'lr: {}, Loss: {:.6f}, Acc:{:.6f}, Semantic loss: {:.6f}, BCE loss: {:.6f}, SB loss: {:.6f}' .format(
                      epoch, num_epoch, i_iter, epoch_iters,
                      batch_time.average(), [x['lr'] for x in optimizer.param_groups], ave_loss.average(),
                      ave_acc.average(), avg_sem_loss.average(), avg_bce_loss.average(),ave_loss.average()-avg_sem_loss.average()-avg_bce_loss.average())
            logging.info(msg)

    writer.add_scalar('train_loss', ave_loss.average(), global_steps)
    writer_dict['train_global_steps'] = global_steps + 1

    # Ritorna la loss media per l'epoca
    return ave_loss.average()

def validate(config, testloader, model, writer_dict):
    model.eval()
    ave_loss = AverageMeter()
    nums = config.MODEL.NUM_OUTPUTS
    confusion_matrix = np.zeros(
        (config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES, nums))
    with torch.no_grad():
        for idx, batch in enumerate(testloader):
            image, label, bd_gts, _, _ = batch
            size = label.size()
            image = image.cuda()
            label = label.long().cuda()
            bd_gts = bd_gts.float().cuda()

            losses, pred, _, _ = model(image, label, bd_gts)
            if not isinstance(pred, (list, tuple)):
                pred = [pred]
            for i, x in enumerate(pred):
                x = F.interpolate(
                    input=x, size=size[-2:],
                    mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS
                )

                confusion_matrix[..., i] += get_confusion_matrix(
                    label,
                    x,
                    size,
                    config.DATASET.NUM_CLASSES,
                    config.TRAIN.IGNORE_LABEL
                )

            if idx % 10 == 0:
                print(idx)

            loss = losses.mean()
            ave_loss.update(loss.item())

    for i in range(nums):
        pos = confusion_matrix[..., i].sum(1)
        res = confusion_matrix[..., i].sum(0)
        tp = np.diag(confusion_matrix[..., i])
        IoU_array = (tp / np.maximum(1.0, pos + res - tp))
        
        ignore_index = 0 
        valid_classes = [i for i in range(config.DATASET.NUM_CLASSES) if i != ignore_index]
        mean_IoU = IoU_array[valid_classes].mean()

        logging.info('{} {} {}'.format(i, IoU_array, mean_IoU))

    writer = writer_dict['writer']
    global_steps = writer_dict['valid_global_steps']
    writer.add_scalar('valid_loss', ave_loss.average(), global_steps)
    writer.add_scalar('valid_mIoU', mean_IoU, global_steps)
    writer_dict['valid_global_steps'] = global_steps + 1
    return ave_loss.average(), mean_IoU, IoU_array


def testval(config, test_dataset, testloader, model,
            sv_dir='./', sv_pred=False):
    model.eval()
    confusion_matrix = np.zeros((config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES))
    with torch.no_grad():
        for index, batch in enumerate(tqdm(testloader)):
            image, label, _, _, name = batch
            size = label.size()
            pred = test_dataset.single_scale_inference(config, model, image.cuda())

            if pred.size()[-2] != size[-2] or pred.size()[-1] != size[-1]:
                pred = F.interpolate(
                    pred, size[-2:],
                    mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS
                )

            confusion_matrix += get_confusion_matrix(
                label,
                pred,
                size,
                config.DATASET.NUM_CLASSES,
                config.TRAIN.IGNORE_LABEL)

            if sv_pred:
                sv_path = os.path.join(sv_dir, 'val_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                test_dataset.save_pred(pred, sv_path, name)

            if index % 100 == 0:
                logging.info('processing: %d images' % index)
                pos = confusion_matrix.sum(1)
                res = confusion_matrix.sum(0)
                tp = np.diag(confusion_matrix)
                IoU_array = (tp / np.maximum(1.0, pos + res - tp))
                mean_IoU = IoU_array.mean()
                logging.info('mIoU: %.4f' % (mean_IoU))

    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)
    pixel_acc = tp.sum()/pos.sum()
    mean_acc = (tp/np.maximum(1.0, pos)).mean()
    IoU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IoU = IoU_array.mean()

    return mean_IoU, IoU_array, pixel_acc, mean_acc


def test(config, test_dataset, testloader, model,
         sv_dir='./', sv_pred=True):
    model.eval()
    with torch.no_grad():
        for _, batch in enumerate(tqdm(testloader)):
            image, size, name = batch
            size = size[0]
            pred = test_dataset.single_scale_inference(
                config,
                model,
                image.cuda())

            if pred.size()[-2] != size[0] or pred.size()[-1] != size[1]:
                pred = F.interpolate(
                    pred, size[-2:],
                    mode='bilinear', align_corners=config.MODEL.ALIGN_CORNERS
                )

            if sv_pred:
                sv_path = os.path.join(sv_dir,'test_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                test_dataset.save_pred(pred, sv_path, name)

def train_adv(config, epoch, num_epoch, epoch_iters, base_lr,
          num_iters, trainloader, targetloader, optimizer_G, optimizer_D, 
          model, discriminator, writer_dict, lambda_adv=0.01, iter_size=4):

    if epoch < 10:
        lambda_adv = 0.01  # all'inizio, il D è debole → serve feedback forte
    else:
        lambda_adv = 0.002  # poi riduci, per non far dominare il D


    # Training mode
    model.train()
    discriminator.train()

    batch_time = AverageMeter()
    ave_loss = AverageMeter()
    ave_acc = AverageMeter()
    avg_sem_loss = AverageMeter()
    avg_bce_loss = AverageMeter()
    
    tic = time.time()
    cur_iters = epoch * epoch_iters
    writer = writer_dict['writer']
    global_steps = writer_dict['train_global_steps']

    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    cumulative_loss_G = 0.0
    cumulative_loss_adv = 0.0
    cumulative_loss_D_src = 0.0
    cumulative_loss_D_trg = 0.0
    count=0
    base_lr_D = 0.0005
    loss_D_src = 0.0 #per eventuale warmup
    loss_D_trg= 0.0
    

    freeze_until_epoch = 0 #freeze del discriminatore


    for i_iter, (batch_source, batch_target) in enumerate(zip(trainloader, targetloader)):

        freeze_D = epoch < freeze_until_epoch
        
        optimizer_G.zero_grad()
        optimizer_D.zero_grad()

        

        #Train G
        # don't accumulate grads in D
        for param in discriminator.parameters():
            param.requires_grad = False

        images_source, labels, bd_gts, _, _ = batch_source
        images_target, _, _, _, _ = batch_target

        images_source = images_source.to(device)
        images_target = images_target.to(device)
        labels = labels.long().to(device)
        bd_gts = bd_gts.float().to(device)


        # ------------------ TRAINING DEL GENERATORE ------------------
        # 1. Forward seg net per dominio sorgente (supervisionato)
        loss_seg1, output_source, _, _ = model(images_source, labels, bd_gts) #retun delle 3 loss sommate ma unsqueezed
        loss_seg1 = torch.squeeze(loss_seg1, 0).mean()
        loss_seg1.backward()
        cumulative_loss_G += loss_seg1.data.cpu().numpy()

        # Forward pass per il dominio target (adversarial)
        _, output_target, _, _ = model(images_target, labels, bd_gts)
        fake_preds = discriminator(F.softmax(output_target[-1], dim=1))

        bce = nn.BCEWithLogitsLoss()
        loss_adv = bce(fake_preds, torch.zeros_like(fake_preds))
        loss_adv = loss_adv * lambda_adv
        #cumulative_loss_G += loss_adv.item()
        loss_adv.backward()
        cumulative_loss_adv += loss_adv.data.cpu().numpy()


        # ------------------ TRAINING DEL DISCRIMINATORE------------------
        if not freeze_D: #per freeze globale 
            
            if 10 <= epoch <= 12:
                for p in discriminator.parameters():
                    p.requires_grad = False
                pass
            else:
                for p in discriminator.parameters():
                    p.requires_grad = True
    
                output_source = [t.detach() for t in output_source]
                output_target = [t.detach() for t in output_target]
        
               
                # train su source
                fake_preds1_d = discriminator(F.softmax(output_source[-1], dim=1)) 
                bce = nn.BCEWithLogitsLoss()
                loss_D_src = bce(fake_preds1_d, torch.zeros_like(fake_preds1_d))
                loss_D_src.backward()
                cumulative_loss_D_src += loss_D_src.data.cpu().numpy()
             
                
                # train su target
                fake_preds1_d_t = discriminator(F.softmax(output_target[-1], dim=1)) 
                bce = nn.BCEWithLogitsLoss()
                loss_D_trg = bce(fake_preds1_d_t, torch.ones_like(fake_preds1_d_t))
                loss_D_trg.backward()
                cumulative_loss_D_trg += loss_D_trg.data.cpu().numpy()
    
                if i_iter % 2 == 0: #aggiorno ogni 2 epoche
                    optimizer_D.step()
        else:
            # se D è freezato, skippa completamente backward e step
            pass
        #--
       
        optimizer_G.step()
        

        # Log
        count+=1
        batch_time.update(time.time() - tic)
        tic = time.time()

        lr_g = adjust_learning_rate(optimizer_G, base_lr, num_iters, i_iter + cur_iters)
        lr_d = adjust_learning_rate(optimizer_D, base_lr_D, num_iters, i_iter + cur_iters, power=1.5)
        

        if i_iter % config.PRINT_FREQ == 0:
            msg = ('Epoch: [{}/{}] Iter:[{}/{}], Time: {:.2f}, lr_generatore: {}, '
                   'Loss_seg: {:.6f}, loss_adv: {:.6f}, loss_D_src: {:.6f}, loss_D_trg: {:.6f}').format(
                      epoch, num_epoch, i_iter, epoch_iters, batch_time.average(),[x['lr'] for x in optimizer_G.param_groups], 
                      loss_seg1, loss_adv, loss_D_src, loss_D_trg
                  )
            logging.info(msg)


    avg_loss_seg   = cumulative_loss_G      / count 
    avg_loss_adv   = cumulative_loss_adv    / count
    avg_loss_D_src = cumulative_loss_D_src  / count
    avg_loss_D_trg = cumulative_loss_D_trg  / count

    writer.add_scalar('Loss/seg',   avg_loss_seg,   global_steps)
    writer.add_scalar('Loss/adv',   avg_loss_adv,   global_steps)
    writer.add_scalar('Loss/D_src', avg_loss_D_src, global_steps)
    writer.add_scalar('Loss/D_trg', avg_loss_D_trg, global_steps)
    writer_dict['train_global_steps'] += 1

    
    # Ritorna la loss media per l'epoca
    final_loss  = ( cumulative_loss_adv + cumulative_loss_D_src + cumulative_loss_D_trg + cumulative_loss_G) / count
    msg = ('Epoch: [{}/{}], loss_sum: {:.6f}').format(epoch, num_epoch, final_loss)
    logging.info(msg)

    return final_loss, avg_loss_seg, avg_loss_adv, avg_loss_D_src, avg_loss_D_trg



### train.py

In [ ]:
# from re import L
# ------------------------------------------------------------------------------
# Modified based on https://github.com/HRNet/HRNet-Semantic-Segmentation
# ------------------------------------------------------------------------------

import argparse
import os
import pprint
import albumentations as A
import logging
import timeit

import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
from tensorboardX import SummaryWriter
from torch import optim
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display
import sys
os.environ["NO_ALBUMENTATIONS_UPDATE"] = "1"


def parse_args():
    parser = argparse.ArgumentParser(description='Train segmentation network')

    parser.add_argument('--cfg',
                        help='experiment configure file name',
                        default="configs/loveda/pidnet_small_loveda.yaml", #file di configurazione da usare
                        type=str)
    parser.add_argument('--seed', type=int, default=304)
    parser.add_argument('--opts',
                        help="Modify config options using the command-line",
                        default=[],
                        nargs=argparse.REMAINDER)

    args, unknown = parser.parse_known_args()
    update_config(config, args) #aggiorna config con tutti i parametri trovati nel file di configurazione

    return args

def plot_metrics(train_loss_history, eval_loss_history, mean_iou_history):
    fig, ax = plt.subplots(3, 1, figsize=(10, 8))

    ax[0].plot(train_loss_history, label='Training Loss', color='blue', marker='o')
    ax[0].plot(eval_loss_history, label='Evaluation Loss', color='orange', marker='x')
    ax[0].set_title('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    ax[0].grid()
    
    ax[1].plot(eval_loss_history, label='Evaluation Loss', color='orange', marker='x')
    ax[1].set_title('Evaluation Loss')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Loss')
    ax[1].legend()
    ax[1].grid()

    ax[2].plot(mean_iou_history, label='Mean IoU', color='green', marker='o')
    ax[2].set_title('Mean IoU')
    ax[2].set_xlabel('Epoch')
    ax[2].set_ylabel('IoU')
    ax[2].legend()
    ax[2].grid()

    plt.tight_layout()
    plt.show()

def plot_class_iou(iou_array, mean_iou, epoch=None):
 
    iou_array = np.array(iou_array)
    num_classes = len(iou_array)
    class_names = [
        'ignored',     # 0
        'background',  # 1
        'building',    # 2
        'road',        # 3
        'water',       # 4
        'barren',      # 5
        'forest',      # 6
        'agriculture'  # 7
    ]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(class_names, iou_array, color='skyblue', edgecolor='black')

    # Linea per la media
    plt.axhline(mean_iou, color='red', linestyle='--', label=f'Mean IoU = {mean_iou:.2f}')

    # Annotazioni sopra le barre
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, height + 0.01, f'{height:.2f}',
                 ha='center', va='bottom', fontsize=9)

    plt.xlabel("Class")
    plt.ylabel("IoU")
    title = "IoU per Classe"
    if epoch is not None:
        title += f" - Epoch {epoch}"
    plt.title(title)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()

    plt.show()

def plot_gan_metrics(loss_seg_hist, loss_adv_hist,loss_D_src_hist, loss_D_trg_hist):
    fig, ax = plt.subplots(3, 1, figsize=(10, 10))

    # 1) loss di segmentazione
    ax[0].plot(loss_seg_hist,  label="Loss Seg (G)", marker="o")
    ax[0].set_title("Segmentation Loss")
    ax[0].set_xlabel("Epoch")
    ax[0].set_ylabel("Loss")
    ax[0].grid();  ax[0].legend()

    # 2) loss adversarial del generatore
    ax[1].plot(loss_adv_hist, label="Loss Adv (G)", marker="x", color="tab:red")
    ax[1].set_title("Adversarial Loss")
    ax[1].set_xlabel("Epoch")
    ax[1].set_ylabel("Loss")
    ax[1].grid();  ax[1].legend()

    # 3) loss del discriminatore (source vs target)
    ax[2].plot(loss_D_src_hist, label="Loss D Src", marker="s", color="tab:green")
    ax[2].plot(loss_D_trg_hist, label="Loss D Trg", marker="d", color="tab:orange")
    ax[2].set_title("Discriminator Loss")
    ax[2].set_xlabel("Epoch")
    ax[2].set_ylabel("Loss")
    ax[2].grid();  ax[2].legend()

    plt.tight_layout()
    plt.show()

def main():
    args = parse_args()

    if args.seed > 0:
        import random
        print('Seeding with', args.seed)
        random.seed(args.seed)
        torch.manual_seed(args.seed)        

    logger, final_output_dir, tb_log_dir = create_logger(config, args.cfg, 'train')
    logger.info(pprint.pformat(args))
    logger.info(config)

    writer_dict = {
        'writer': SummaryWriter(tb_log_dir),
        'train_global_steps': 0,
        'valid_global_steps': 0,
    }

    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")

    if torch.cuda.is_available():
        # cudnn related setting
        cudnn.benchmark = config.CUDNN.BENCHMARK
        cudnn.deterministic = config.CUDNN.DETERMINISTIC
        cudnn.enabled = config.CUDNN.ENABLED
        gpus = list(config.GPUS)
        if torch.cuda.device_count() != len(gpus):
            print("The gpu numbers do not match!")
            return 0
    gpus = list(config.GPUS)
    
    imgnet = 'imagenet' in config.MODEL.PRETRAINED

    
    model = get_seg_model(config, imgnet_pretrained=imgnet)
 
    batch_size = config.TRAIN.BATCH_SIZE_PER_GPU * len(gpus)
    # prepare data
    #crop_size = (config.TRAIN.IMAGE_SIZE[1], config.TRAIN.IMAGE_SIZE[0])
    crop_size = (1024, 1024)

    train_trasform = None

    if config.TRAIN.AUGMENTATION.ENABLE:
        list_augmentations = []
        if config.TRAIN.AUGMENTATION.TECHNIQUES.RANDOM_CROP:
            list_augmentations.append(A.RandomResizedCrop(1024, 1024, p=0.5))
        if config.TRAIN.AUGMENTATION.TECHNIQUES.HORIZONTAL_FLIP:
            list_augmentations.append(A.HorizontalFlip(p=0.5))
        if config.TRAIN.AUGMENTATION.TECHNIQUES.COLOR_JITTER:
            list_augmentations.append(A.ColorJitter(p=0.5))
        if config.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_BLUR:
            list_augmentations.append(A.GaussianBlur(p=0.5))
        if config.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_NOISE:
            list_augmentations.append(A.GaussNoise(var_limit=(100, 1000), p=1.0))
        if len(list_augmentations) != 0:
            train_trasform = A.Compose(list_augmentations)

    
    print("train with standard dataset")
    #The eval() function evaluates the specified expression, if the expression is a legal Python statement, it will be executed.
    train_dataset = LoveDA(
                        root=config.DATASET.ROOT,
                        list_path=config.DATASET.TRAIN_SET,
                        num_classes=config.DATASET.NUM_CLASSES,
                        multi_scale=config.TRAIN.MULTI_SCALE,
                        flip=config.TRAIN.FLIP,
                        ignore_label=config.TRAIN.IGNORE_LABEL,
                        base_size=config.TRAIN.BASE_SIZE,
                        crop_size=crop_size,
                        scale_factor=config.TRAIN.SCALE_FACTOR,
                        enable_augmentation=True,
                        horizontal_flip=config.TRAIN.AUGMENTATION.TECHNIQUES.HORIZONTAL_FLIP,
                        gaussian_blur=config.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_BLUR,
                        transform=train_trasform)
    
    

    trainloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=config.TRAIN.SHUFFLE,
        num_workers=config.WORKERS,
        pin_memory=False,
        drop_last=True)
    

    targetloader = None
    if config.TRAIN.DACS.ENABLE or config.TRAIN.GAN.ENABLE:
        target_dataset = LoveDA(
                            root=config.DATASET.ROOT, 
                            list_path=config.DATASET.TARGET_SET,
                            num_classes=config.DATASET.NUM_CLASSES, 
                            multi_scale=config.TRAIN.MULTI_SCALE,
                            flip=config.TRAIN.FLIP, 
                            enable_augmentation=True,
                            ignore_label=config.TRAIN.IGNORE_LABEL,
                            base_size=config.TRAIN.BASE_SIZE,
                            crop_size=crop_size, 
                            scale_factor=config.TRAIN.SCALE_FACTOR,
                            horizontal_flip=config.TRAIN.AUGMENTATION.TECHNIQUES.HORIZONTAL_FLIP,
                            gaussian_blur=config.TRAIN.AUGMENTATION.TECHNIQUES.GAUSSIAN_BLUR,
                            transform=train_trasform)

        targetloader = torch.utils.data.DataLoader(
            target_dataset, 
            batch_size=batch_size,
            shuffle=config.TRAIN.SHUFFLE,
            num_workers=config.WORKERS, 
            pin_memory=False, 
            drop_last=True)


    test_size = (config.TEST.IMAGE_SIZE[1], config.TEST.IMAGE_SIZE[0])
    test_dataset = LoveDA(
                        root=config.DATASET.ROOT,
                        list_path=config.DATASET.TEST_SET,
                        num_classes=config.DATASET.NUM_CLASSES,
                        multi_scale=False,
                        flip=False,
                        ignore_label=config.TRAIN.IGNORE_LABEL,
                        base_size=config.TEST.BASE_SIZE,
                        crop_size=test_size)

    testloader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.TEST.BATCH_SIZE_PER_GPU * len(gpus),
        shuffle=False,
        num_workers=config.WORKERS,
        pin_memory=False)

    # criterion
    if config.LOSS.USE_OHEM:
        sem_criterion = OhemCrossEntropy(ignore_label=config.TRAIN.IGNORE_LABEL,
                                        thres=config.LOSS.OHEMTHRES,
                                        min_kept=config.LOSS.OHEMKEEP,
                                        weight=train_dataset.class_weights)
    else:
        sem_criterion = CrossEntropy(ignore_label=config.TRAIN.IGNORE_LABEL,
                                    weight=train_dataset.class_weights)

    bd_criterion = BondaryLoss()
    
    
    model = FullModel(model, sem_criterion, bd_criterion)

    if torch.cuda.is_available():
        model = nn.DataParallel(model, device_ids=gpus).cuda() #per noi inutile
    else:
        model = model.to(device)

    # optimizer
    if config.TRAIN.OPTIMIZER == 'sgd':
        params_dict = dict(model.named_parameters())
        params = [{'params': list(params_dict.values()), 'lr': config.TRAIN.LR}]

        optimizer = torch.optim.SGD(params,
                                lr=config.TRAIN.LR,
                                momentum=config.TRAIN.MOMENTUM,
                                weight_decay=config.TRAIN.WD,
                                nesterov=config.TRAIN.NESTEROV,
                                )
        #optimizer = Optimizer(model=model.module, loss=bd_criterion,
        #                  lr0=1e-2, momentum=0.9, wd=5e-4,
        #                  warmup_steps=960, warmup_start_lr=1e-5,
        #                  max_iter=len(trainloader)*20,
        #                  power=0.9)
    else:
        raise ValueError('Only Support SGD optimizer')

    epoch_iters = int(train_dataset.__len__() / config.TRAIN.BATCH_SIZE_PER_GPU / len(gpus))
        
    best_mIoU = 0
    last_epoch = 0
    flag_rm = config.TRAIN.RESUME
    if config.TRAIN.RESUME:
        model_state_file = os.path.join(final_output_dir, 'checkpoint.pth.tar')
        if os.path.isfile(model_state_file):
            print('-'*60)
            checkpoint = torch.load(model_state_file, map_location={'cuda:0': 'cpu'})
            best_mIoU = checkpoint['best_mIoU']
            last_epoch = checkpoint['epoch']
            dct = checkpoint['state_dict']
            
            model.module.model.load_state_dict({k.replace('model.', ''): v for k, v in dct.items() if k.startswith('model.')})
            optimizer.load_state_dict(checkpoint['optimizer'])
            logger.info("=> loaded checkpoint (epoch {})".format(checkpoint['epoch']))

    start = timeit.default_timer()
    end_epoch = config.TRAIN.END_EPOCH
    num_iters = config.TRAIN.END_EPOCH * epoch_iters
    real_end = 120+1 if 'camvid' in config.DATASET.TRAIN_SET else end_epoch
    

    train_loss_history = []
    eval_loss_history = []
    mean_iou_history = []
    hist_seg, hist_adv, hist_D_src, hist_D_trg = [], [], [], []

    for epoch in range(last_epoch, real_end):

        current_trainloader = trainloader
        if current_trainloader.sampler is not None and hasattr(current_trainloader.sampler, 'set_epoch'):
            current_trainloader.sampler.set_epoch(epoch)

        if config.TRAIN.GAN.ENABLE:
            
            discriminator1 = FCDiscriminator(num_classes=8).to(device)
            optimizer_G = optimizer
            optimizer_D1 = optim.Adam(discriminator1.parameters(), lr=0.0005, betas=(0.9, 0.99)) #given by the paper

            train_loss, avg_seg, avg_adv, avg_D_src, avg_D_trg =train_adv(config, epoch, config.TRAIN.END_EPOCH, epoch_iters, config.TRAIN.LR, num_iters, trainloader, targetloader, optimizer_G, optimizer_D1, model, discriminator1, writer_dict)
            hist_seg.append(avg_seg)
            hist_adv.append(avg_adv)
            hist_D_src.append(avg_D_src)
            hist_D_trg.append(avg_D_trg)
            print("Loss Summary:  Loss_Seg (G): {:.6f}, Loss_Adv (G): {:.6f}, Loss_D_Src: {:.6f}, Loss_D_Trg: {:.6f}".format(avg_seg, avg_adv, avg_D_src, avg_D_trg))

        else:
            train_loss=train(config, epoch, config.TRAIN.END_EPOCH, 
                  epoch_iters, config.TRAIN.LR, num_iters,
                  trainloader, optimizer, model, writer_dict, targetloader=targetloader)

        train_loss_history.append(train_loss)

        if flag_rm == 1 or (epoch % 5 == 0 and epoch < real_end - 100) or (epoch >= real_end - 100):
            valid_loss, mean_IoU, IoU_array = validate(config, testloader, model, writer_dict)
            eval_loss_history.append(valid_loss)
            mean_iou_history.append(mean_IoU)

        if flag_rm == 1:
            flag_rm = 0

        #if epoch % 5 == 0 or epoch == real_end - 1:
        plot_metrics(train_loss_history, eval_loss_history, mean_iou_history)
        if config.TRAIN.GAN.ENABLE:
            plot_gan_metrics(hist_seg, hist_adv, hist_D_src, hist_D_trg)
        plot_class_iou(iou_array=IoU_array, mean_iou=mean_IoU, epoch=epoch)
        


        logger.info('=> saving checkpoint to {}'.format(
            final_output_dir + 'checkpoint.pth.tar'))
        torch.save({
            'epoch': epoch+1,
            'best_mIoU': best_mIoU,
            'state_dict': model.module.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, os.path.join(final_output_dir,'checkpoint.pth.tar'))
        if mean_IoU > best_mIoU:
            best_mIoU = mean_IoU
            torch.save(model.module.state_dict(),
                    os.path.join(final_output_dir, 'best.pt'))
        msg = 'Loss: {:.3f}, MeanIU: {: 4.4f}, Best_mIoU: {: 4.4f}'.format(
                    valid_loss, mean_IoU, best_mIoU)
        logging.info(msg)
        logging.info(IoU_array)



    torch.save(model.module.state_dict(),
            os.path.join(final_output_dir, 'final_state.pt'))

    writer_dict['writer'].close()
    end = timeit.default_timer()
    logger.info('Hours: %d' % int((end-start)/3600))
    logger.info('Done')

    plt.ioff()
    plt.show()
if __name__ == '__main__':
    main()

Namespace(cfg='configs/loveda/pidnet_small_loveda.yaml', seed=304, opts=[])
AUTO_RESUME: False
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  DATASET: loveda
  EXTRA_TRAIN_SET: 
  NUM_CLASSES: 8
  ROOT: data/
  TARGET_SET: list/loveda/rural/train.lst
  TEST_SET: list/loveda/urban_rural/val.lst
  TRAIN_SET: list/loveda/urban_rural/train.lst
GPUS: (0,)
LOG_DIR: log
LOSS:
  BALANCE_WEIGHTS: [0.4, 1.0]
  CLASS_BALANCE: False
  OHEMKEEP: 131072
  OHEMTHRES: 0.9
  SB_WEIGHTS: 1.0
  USE_OHEM: True
MODEL:
  ALIGN_CORNERS: True
  NAME: pidnet_small
  NUM_OUTPUTS: 2
  PRETRAINED: pretrained_models/imagenet/PIDNet_S_ImageNet.pth.tar
OUTPUT_DIR: output
PIN_MEMORY: True
PRINT_FREQ: 10
TEST:
  BASE_SIZE: 1024
  BATCH_SIZE_PER_GPU: 6
  FLIP_TEST: False
  IMAGE_SIZE: [1024, 1024]
  MODEL_FILE: output/loveda/pidnet_small_loveda/final_state.pt
  MULTI_SCALE: False
  OUTPUT_INDEX: 1
TRAIN:
  AUGMENTATION:
    ENABLE: True
    PROBABILITY: 0.5
    TECHNIQUES:
      COLOR_JITTER

Seeding with 304
=> creating output/loveda/pidnet_small_loveda/gan_cj
=> creating log/loveda/pidnet_small/pidnet_small_loveda_2025-05-24-10-32
Using CUDA


/tmp/ipykernel_161/4253090261.py:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state = torch.load(cfg.MODEL.PRETRAINED, map_location='cpu')['state_dict']
Att

train with standard dataset


Epoch: [0/20] Iter:[0/192], Time: 50.20, lr_generatore: [0.005], Loss_seg: 12.420799, loss_adv: 0.006964, loss_D_src: 0.696476, loss_D_trg: 0.689862
Epoch: [0/20] Iter:[10/192], Time: 5.67, lr_generatore: [0.004988279722662096], Loss_seg: 6.692202, loss_adv: 0.006715, loss_D_src: 0.670698, loss_D_trg: 0.715478
Epoch: [0/20] Iter:[20/192], Time: 3.56, lr_generatore: [0.0049765563847963806], Loss_seg: 5.239687, loss_adv: 0.007074, loss_D_src: 0.705183, loss_D_trg: 0.680651
Epoch: [0/20] Iter:[30/192], Time: 2.81, lr_generatore: [0.004964829977588641], Loss_seg: 3.403477, loss_adv: 0.006508, loss_D_src: 0.649604, loss_D_trg: 0.738825
Epoch: [0/20] Iter:[40/192], Time: 2.42, lr_generatore: [0.004953100492176077], Loss_seg: 2.860073, loss_adv: 0.007009, loss_D_src: 0.700411, loss_D_trg: 0.685498
Epoch: [0/20] Iter:[50/192], Time: 2.18, lr_generatore: [0.0049413679196469015], Loss_seg: 3.381996, loss_adv: 0.006719, loss_D_src: 0.668850, loss_D_trg: 0.715194
Epoch: [0/20] Iter:[60/192], Time:

...

0 [0.         0.39059344 0.24318124 0.13887818 0.26599414 0.10050989
 0.10928571 0.25512575] 0.2147954794914581
1 [0.         0.5279234  0.31785875 0.29347164 0.30257809 0.11993598
 0.13484316 0.36415744] 0.2943954945501798


<Figure size 1000x800 with 3 Axes>

<Figure size 1000x1000 with 3 Axes>

<Figure size 1000x600 with 1 Axes>

=> saving checkpoint to output/loveda/pidnet_small_loveda/gan_cjcheckpoint.pth.tar
Loss: 8.926, MeanIU:  0.2944, Best_mIoU:  0.3149
[0.         0.5279234  0.31785875 0.29347164 0.30257809 0.11993598
 0.13484316 0.36415744]
Hours: 2
Done


In [24]:
import shutil

# Zippa la cartella dei log
shutil.make_archive('/kaggle/working/log', 'zip', '/kaggle/working/log')
shutil.make_archive('/kaggle/working/output', 'zip', '/kaggle/working/output')


'/kaggle/working/output.zip'